# Algorithm for data setup
The purpose of the algorithm bellow is to normalize the inserted .csv file and return 2 new .csv files. One is for training and the other for test.

## Importing data
The code down below imports and cleans the data from a csv.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dataFrame = pd.read_csv("Evadidos_concluintes_307.csv", sep=',')
file = open("Evadidos_concluintes_307.csv", "r")

In [3]:
#apaga as seguintes colunas
dataFrame.drop(['COD_CURSO'], axis=1, inplace=True)
dataFrame.drop(['NOME_CURSO'], axis=1, inplace=True)
dataFrame.drop(['ID_TURMA'], axis=1, inplace=True)
dataFrame.drop(['ID_DISCIPLINA'], axis=1, inplace=True)

In [4]:
dataFrame = dataFrame.sort_values(by=["ANO_INGRESSO", "ID_CURSO_ALUNO"])

In [5]:
#retira os espaços no inicio e final da string
dataFrame["COD_DISCIPLINA"] = dataFrame["COD_DISCIPLINA"].str.strip()
#transforma floats como 1.992 em numeros inteiros (1992)
dataFrame["ID_CURSO_ALUNO"] = round(dataFrame["ID_CURSO_ALUNO"]*1000)
dataFrame["ANO_INGRESSO"] = round(dataFrame["ANO_INGRESSO"]*1000)
dataFrame["ANO_CONC_DISC"] = round(dataFrame["ANO_CONC_DISC"]*1000)
dataFrame["ANO_EVASAO"] = round(dataFrame["ANO_EVASAO"]*1000)

In [6]:
#substitui itens iguais a 4 por "Formou" e itens diferentes de 4 por "NaoFormou"
dataFrame["FORMA_EVASAO_ITEM"] = np.where(dataFrame["FORMA_EVASAO_ITEM"] == 4, 'Formou', 'NaoFormou')

In [7]:
dataFrame.head()

,ANO_INGRESSO,PERIODO_INGRESSO,ID_CURSO_ALUNO,COD_DISCIPLINA,ANO_CONC_DISC,PERIODO_CONC_DISC,PERIODO_IDEAL,FORMA_EVASAO_ITEM,ANO_EVASAO,PERIODO_EVASAO
0,1992.0,1,34506.0,ELC104,1992.0,1,1,Formou,1997.0,2
1,1992.0,1,34506.0,ELC105,1992.0,1,1,Formou,1997.0,2
2,1992.0,1,34506.0,LTE222,1992.0,1,1,Formou,1997.0,2
3,1992.0,1,34506.0,MTM101,1992.0,1,1,Formou,1997.0,2
4,1992.0,1,34506.0,MTM144,1992.0,1,1,Formou,1997.0,2


## Normalizing data
The code down below separates the data into three dictionaries:
- disciplina (**COD_DISCIPLINA**, PERIODO_IDEAL)
- aluno (**ID_CURSO_ALUNO**, ANO_INGRESSO, PERIODO_INGRESSO, ANO_EVASAO, PERIODO_EVASAO, FORMA_EVASAO_ITEM,)
- conclusao (**COD_DISCIPLINA**, **ID_CURSO_ALUNO**, PERIODO_CONC_DISC, ANO_CONC_DISC)

In [8]:
#faz uma lista com todas as disciplinas
disciplina = []
for codigoDisciplina in dataFrame["COD_DISCIPLINA"]:
    if (codigoDisciplina not in disciplina):
        disciplina.append(codigoDisciplina)

In [9]:
#faz um hash com entrada ID_CURSO_ALUNO e saídas mostradas abaixo
aluno = {}
for codigoAluno, anoIngresso, periodoIngresso, anoEvasao, periodoEvasao, tipoEvasao in zip(dataFrame["ID_CURSO_ALUNO"], dataFrame["ANO_INGRESSO"], dataFrame["PERIODO_INGRESSO"], dataFrame["ANO_EVASAO"], dataFrame["PERIODO_EVASAO"], dataFrame["FORMA_EVASAO_ITEM"]):
    aluno[codigoAluno] = [anoIngresso, periodoIngresso, anoEvasao, periodoEvasao, tipoEvasao]

In [10]:
disciplinasCursadasPorAluno = {}
for codigoAluno in dataFrame["ID_CURSO_ALUNO"]:
    disciplinasCursadasPorAluno[codigoAluno] = []
    for codigoDisciplina in dataFrame["COD_DISCIPLINA"].loc[dataFrame["ID_CURSO_ALUNO"] == codigoAluno]:    
        disciplinasCursadasPorAluno[codigoAluno].append(codigoDisciplina)

In [11]:
#faz um hash com entradas ID_CURSO_ALUNO e COD_DISCIPLINA e saídas mostradas abaixo
conclusao = {}
for codigoAluno in dataFrame["ID_CURSO_ALUNO"]:
    conclusao[codigoAluno] = {} 
    for codigoDisciplina, periodoConclusao, anoConclusao in zip(dataFrame["COD_DISCIPLINA"].loc[dataFrame["ID_CURSO_ALUNO"] == codigoAluno], dataFrame["PERIODO_CONC_DISC"].loc[dataFrame["ID_CURSO_ALUNO"] == codigoAluno], dataFrame["ANO_CONC_DISC"].loc[dataFrame["ID_CURSO_ALUNO"] == codigoAluno]):
        conclusao[codigoAluno][codigoDisciplina] = [anoConclusao, periodoConclusao]

In [12]:
#disciplina

In [13]:
#aluno

In [14]:
#disciplinasCursadasPorAluno

In [15]:
#conclusao

## Creating new DataFrame
The code down below uses the dictionaries created above to make a new DataFrame that uses the student identification (*ID_CURSO_ALUNO*) as ID and transforms each subject (*disciplina*) in one column.

In [16]:
#A primeira coisa a ser feita é criar um dataframe com o codigo do aluno como identificador
newDataFrame = pd.DataFrame(aluno.values(), columns=["ANO_INGRESSO", "PERIODO_INGRESSO", "ANO_EVASAO", "PERIODO_EVASAO", "FORMA_EVASAO_ITEM"])
newDataFrame.insert(0, "ID_CURSO_ALUNO", aluno.keys())
for i in disciplina:
    newDataFrame[i] = np.nan

In [17]:
def studentConclusionSemester(codigoAluno, codigoDisciplina):    
    #se periodos de conclusão e ingresso forem iguais
    if conclusao[codigoAluno][codigoDisciplina][1] >= aluno[codigoAluno][1]:
        #(ano de conclusão - ano de ingresso)*2 + (periodo de conclusão - periodo de ingresso) + 1
        return (conclusao[codigoAluno][codigoDisciplina][0] - aluno[codigoAluno][0]) * 2 + aluno[codigoAluno][1] - conclusao[codigoAluno][codigoDisciplina][1] + 1
    #se periodo de conclusao for menor que de ingresso
    elif conclusao[codigoAluno][codigoDisciplina][1] < aluno[codigoAluno][1]:    
        #(ano de conclusão - ano de ingresso)*2
        return (conclusao[codigoAluno][codigoDisciplina][0] - aluno[codigoAluno][0]) * 2

### Subject conclusion mesure
The function below sets difference between the conclusion semester and the ingress student semester.

In [18]:
def differenceBetweenIdealAndConclusionSemesters(codigoAluno, codigoDisciplina):
    if conclusao.get(codigoAluno) == None:
        return np.nan
    elif conclusao[codigoAluno].get(codigoDisciplina) == None:
        return np.nan
    else:
        periodoIdeal = dataFrame["PERIODO_IDEAL"].loc[(dataFrame["ID_CURSO_ALUNO"] == codigoAluno) & (dataFrame["COD_DISCIPLINA"] == codigoDisciplina)].get_values()
        if periodoIdeal[0] == 99:
            return 0
        return periodoIdeal[0] - studentConclusionSemester(codigoAluno, codigoDisciplina)

In [19]:
for codigoAluno in newDataFrame["ID_CURSO_ALUNO"]:
    for codigoDisciplina in disciplinasCursadasPorAluno[codigoAluno]:
        newDataFrame.loc[newDataFrame["ID_CURSO_ALUNO"] == codigoAluno, codigoDisciplina] = differenceBetweenIdealAndConclusionSemesters(codigoAluno, codigoDisciplina)

In [20]:
newDataFrame = newDataFrame.sort_values(by=["ANO_INGRESSO", "ID_CURSO_ALUNO"])

In [21]:
newDataFrame.head()

,ID_CURSO_ALUNO,ANO_INGRESSO,PERIODO_INGRESSO,ANO_EVASAO,PERIODO_EVASAO,FORMA_EVASAO_ITEM,ELC104,ELC105,LTE222,MTM101,...,MTM1003,DCT1011,ELC1093,ELC1096,ELC1104,EDE1114,ELC1108,DPADI0029,DCOM1000,ELC1074
0,34506.0,1992.0,1,1997.0,2,Formou,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34507.0,1992.0,1,1995.0,2,Formou,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34509.0,1992.0,1,1995.0,2,Formou,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34510.0,1992.0,1,1995.0,2,Formou,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34511.0,1992.0,1,1995.0,2,Formou,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Separating the data
Next it will be choosed the data that is going to be used for training and test based on the student entry year.

In [22]:
trainingDataFrame = newDataFrame.loc[(newDataFrame["ANO_INGRESSO"] >= 1996) & (newDataFrame["ANO_EVASAO"] < 2006)]
trainingDataFrame

,ID_CURSO_ALUNO,ANO_INGRESSO,PERIODO_INGRESSO,ANO_EVASAO,PERIODO_EVASAO,FORMA_EVASAO_ITEM,ELC104,ELC105,LTE222,MTM101,...,MTM1003,DCT1011,ELC1093,ELC1096,ELC1104,EDE1114,ELC1108,DPADI0029,DCOM1000,ELC1074
122,43889.0,1996.0,1,1999.0,2,NaoFormou,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,43890.0,1996.0,1,1999.0,2,Formou,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,43891.0,1996.0,1,2002.0,1,NaoFormou,NaN,NaN,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,43892.0,1996.0,1,2003.0,1,NaoFormou,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,43893.0,1996.0,1,1999.0,2,Formou,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,43894.0,1996.0,1,1999.0,2,Formou,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,43895.0,1996.0,1,1999.0,2,Formou,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,43896.0,1996.0,1,2001.0,2,NaoFormou,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,43897.0,1996.0,1,2000.0,2,Formou,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,43898.0,1996.0,1,2002.0,1,Formou,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
testDataFrame = newDataFrame.loc[(newDataFrame["ANO_INGRESSO"] < 2006) & (newDataFrame["ANO_EVASAO"] >= 2006)]
testDataFrame

,ID_CURSO_ALUNO,ANO_INGRESSO,PERIODO_INGRESSO,ANO_EVASAO,PERIODO_EVASAO,FORMA_EVASAO_ITEM,ELC104,ELC105,LTE222,MTM101,...,MTM1003,DCT1011,ELC1093,ELC1096,ELC1104,EDE1114,ELC1108,DPADI0029,DCOM1000,ELC1074
207,49510.0,1998.0,1,2006.0,1,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,128157.0,1998.0,1,2006.0,2,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,51696.0,1999.0,1,2008.0,2,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,51707.0,1999.0,1,2009.0,1,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,51709.0,1999.0,1,2006.0,1,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,52443.0,1999.0,1,2006.0,2,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,53134.0,1999.0,1,2006.0,1,Formou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,54061.0,2000.0,1,2006.0,1,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,54062.0,2000.0,1,2006.0,2,Formou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,54069.0,2000.0,1,2007.0,2,NaoFormou,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#train, test = train_test_split(newDataFrame)

In [25]:
#newDataFrame.to_csv("new_data_frame.csv", index = False)
trainingDataFrame.to_csv("training_data_frame.csv", index = False)
testDataFrame.to_csv("test_data_frame.csv", index = False)